In [1]:
# importation de package necessaire
import pandas as pd
import numpy as np
from datetime import datetime
import warnings 
warnings.filterwarnings("ignore")
import pyodbc
import csv

In [2]:
df_geography = pd.read_csv("dataset/geography1.csv")
df_geography.head()

,Geo_Key,Country,Region,Department,District
0,1,Sénégal,Dakar,Dakar,Gueule Tapee Fass Colobane
1,2,Sénégal,Dakar,Dakar,Fann Point E Amitie
2,3,Sénégal,Dakar,Dakar,Goree
3,4,Sénégal,Dakar,Dakar,Medina
4,5,Sénégal,Dakar,Dakar,Plateau


In [3]:
# connexion à sql server
server = 'DESKTOP-NNQ794M\SQLM2BI'  # Nom ou adresse IP du serveur SQL Server
database = 'Beauty_treasure_DWH'  # Nom de la base de données
username = 'clarisse'  # Nom d'utilisateur SQL Server
password = 'Clarisse.'  # Mot de passe SQL Server

conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

In [4]:
def lst_column(df) :
    lst_column = df.columns.tolist()
    col = ''
    for i in (lst_column):
        col = col + i + ","
    return col[:-1]

def interrogation(string):
    count = ''
    for char in string:
        if char == ',':
            count += '?,'
    return count+'?'

In [5]:
# chargement dans la table store
def loading_in_table(df, table_name, file_name):
    csv_file_path = "dataset/"+file_name + '.csv'
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # Ouverture du fichier CSV
        with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile, delimiter=",")
            next(csv_reader)  # Ignore l'en-tête du fichier CSV
#             cursor.execute(f"DELETE FROM {table_name}")
            # Itération à travers chaque ligne du fichier CSV
            lst_col = lst_column(df)
            lst_interro = interrogation(lst_col)
            for row in csv_reader:
#                 print(row)
                query = f"INSERT INTO {table_name}({lst_col}) VALUES ({lst_interro})"
                cursor.execute(query, row)
                

        # Validation des changements
        conn.commit()
        print("Chargement des données dans la table avec succès.")

        # Fermeture du curseur et de la connexion
        cursor.close()
        conn.close()

    except pyodbc.Error as e:
        print(f"Erreur lors de la connexion à la base de données : {e}")

In [ ]:
df_geography = pd.read_csv("dataset/geography1.csv")
loading_in_table(df_geography, "DWH.Dim_Geographies", "geography1")